In [ ]:
import os
import subprocess
import time
from pathlib import Path
from itertools import islice
from tqdm import tqdm
import shutil
import os
from pathlib import Path

In [ ]:
APK_DIR = "/scratch/rhm4nj/ml-sec/data/apks"
LOG_DIR = "/scratch/rhm4nj/ml-sec/logs/apk_jobs"
SCRIPT_PATH = "/scratch/rhm4nj/ml-sec/process_apk.py"  # your main script
BATCH_SIZE = 25
MAX_JOBS = 100  # parallel limit
MAX_RUNS = 1e10
SLURM_TIME = "00:15:00"
MEM = "2G"

if os.path.exists(LOG_DIR): shutil.rmtree(LOG_DIR)
Path(LOG_DIR).mkdir(parents=True, exist_ok=True)

if os.path.exists("stop"):
    os.remove("stop")

def active_job_count():
    cmd = ["squeue", "-h", "-u", os.getenv("USER")]
    result = subprocess.run(cmd, capture_output=True, text=True)
    return len(result.stdout.strip().splitlines())

def batch(iterable, size):
    it = iter(iterable)
    while True:
        chunk = list(islice(it, size))
        if not chunk:
            break
        yield chunk

def write_job_script(batch_id, apk_paths):
    script_fp = f"{LOG_DIR}/apk_batch_{batch_id}.sh"
    with open(script_fp, "w") as f:
        f.write(f"""#!/bin/bash
#SBATCH --account=cral
#SBATCH --partition=standard
#SBATCH --job-name=apk_batch_{batch_id}
#SBATCH --output={LOG_DIR}/batch_{batch_id}.out
#SBATCH --error={LOG_DIR}/batch_{batch_id}.err
#SBATCH --time={SLURM_TIME}
#SBATCH --mem={MEM}
#SBATCH --cpus-per-task=1

""")
        for apk in apk_paths:
            f.write(f"python {SCRIPT_PATH} {apk}\n")
    return script_fp

def submit_apk_jobs():
    all_apks = sorted([str(p) for p in Path(APK_DIR).glob("*.apk")])
    batches = list(batch(all_apks, BATCH_SIZE))
    for i, apk_batch in tqdm(enumerate(batches), total=min(len(batches), MAX_RUNS), desc="Submitting jobs"):
        if active_job_count() > MAX_RUNS: break
        while active_job_count() >= MAX_JOBS:
            time.sleep(10)
        script = write_job_script(i, apk_batch)
        subprocess.run(["sbatch", script])

        if "stop" in os.listdir("."):
            print("manual stop")
            break

    print("All jobs submitted.")

submit_apk_jobs()
